# SEEK read API

In this case use the LiSyM instance  
https://seek.lisym.org/  
SEEK v1.6.2

adapted from Wolfgang Müller, HITS

In [16]:
SEEK_URL = "https://seek.lisym.org/"

# Read available publications from SEEK instance

In [18]:
import urllib.request
import json
import datetime

def getAllPublications():
    """ Get JSON for all publications."""
    url = SEEK_URL + 'publications.json'
    req = urllib.request.Request(url)
    r = urllib.request.urlopen(req).read()
    cont = json.loads(r.decode('utf-8'))

    return cont['data']

def getPublication(id):
    url = SEEK_URL + "publications/%s.json" % id
    fullPublicationRequest = urllib.request.Request(url)
    fullPublicationText = urllib.request.urlopen(fullPublicationRequest).read()
    fullPublicationJson = json.loads(fullPublicationText.decode('utf-8'))
    return fullPublicationJson['data']

def parsePublication(data):
    attributes = data['attributes']
    authors = attributes['authors']
    title = attributes['title']
    journal = attributes['journal']
    citation = attributes['citation']
    published_date = attributes['published_date']
    projects = data['relationships']['projects']

    return ({
          'title': title,
          'authors': authors,
          'citation': citation,
          'journal': journal,
          'published_date': published_date,
          'projects': projects
    })

allPublicationsData = getAllPublications()
counter = 0
publications = []
for item in allPublicationsData:
    # print(item)
    counter += 1
    id = item["id"]
    fullPublicationData = getPublication(id)
    
    publications.append(parsePublication(fullPublicationData))
print(counter)
# now "publications" contains a list of hashes with publication information

98


In [19]:
print(publications[0])

{'authors': ['F. Schreiber', 'G. D. Bader', 'P. Gleeson', 'Martin Golebiewski', 'M. Hucka', 'N. Le Novere', 'C. Myers', 'D. Nickerson', 'B. Sommer', 'D. Walthemath'], 'title': 'Specifications of Standards in Systems and Synthetic Biology: Status and Developments in 2016', 'published_date': '2017-02-12', 'citation': 'J Integr Bioinform. 2016 Dec 18;13(3):289. doi: 10.2390/biecoll-jib-2016-289.', 'journal': 'J Integr Bioinform', 'projects': {'data': [{'id': '2', 'type': 'projects'}]}}


In [21]:
import pandas as pd
df_pub = pd.DataFrame(publications)
df_pub = df_pub[["title", "journal", "published_date", "citation", "authors", "projects"]]

# save as excel file
writer = pd.ExcelWriter('./results/publications.xlsx', engine='xlsxwriter')
df_pub.to_excel(writer, sheet_name='Publications', index=False)
writer.save()

df_pub.head(10)

,title,journal,published_date,citation,authors,projects
0,Specifications of Standards in Systems and Syn...,J Integr Bioinform,2017-02-12,J Integr Bioinform. 2016 Dec 18;13(3):289. doi...,"[F. Schreiber, G. D. Bader, P. Gleeson, Martin...","{'data': [{'id': '2', 'type': 'projects'}]}"
1,FAIRDOMHub: a repository and collaboration env...,Nucleic Acids Res,2016-12-03,Nucleic Acids Res. 2017 Jan 4;45(D1):D404-D407...,"[K. Wolstencroft, Olga Krebs, J. L. Snoep, N. ...","{'data': [{'id': '2', 'type': 'projects'}]}"
2,"The Human Physiome: how standards, software an...",Interface Focus,2016-04-07,Interface Focus. 2016 Apr 6;6(2):20150103. doi...,"[D. Nickerson, K. Atalag, B. de Bono, J. Geige...","{'data': [{'id': '2', 'type': 'projects'}]}"
3,"SABIO-RK, von Daten in der Publikation zur Suc...",Datenbank Spektrum,2017-03-01,Datenbank Spektrum 17(1) : 21,"[Wolfgang Müller, Meik Bittkowski, Martin Gole...","{'data': [{'id': '2', 'type': 'projects'}]}"
4,L1 regularization facilitates detection of cel...,Bioinformatics,2016-09-03,Bioinformatics. 2016 Sep 1;32(17):i718-i726. d...,"[B. Steiert, J. Timmer, C. Kreutz]","{'data': [{'id': '5', 'type': 'projects'}]}"
5,Identification of Cell Type-Specific Differenc...,PLoS Comput Biol,2016-08-06,PLoS Comput Biol. 2016 Aug 5;12(8):e1005049. d...,"[R. Merkle, B. Steiert, F. Salopiata, S. Depne...","{'data': [{'id': '5', 'type': 'projects'}]}"
6,Driving the Model to Its Limit: Profile Likeli...,PLoS One,2016-09-03,PLoS One. 2016 Sep 2;11(9):e0162366. doi: 10.1...,"[T. Maiwald, H. Hass, B. Steiert, J. Vanlier, ...","{'data': [{'id': '5', 'type': 'projects'}]}"
7,Model-guided identification of a therapeutic s...,Journal of Hepatology,2016-04-01,Journal of Hepatology 64(4) : 860,"[Ahmed Ghallab, Géraldine Cellière, Sebastian ...","{'data': [{'id': '7', 'type': 'projects'}]}"
8,In vivo imaging of systemic transport and elim...,Arch Toxicol,2017-03-01,Arch Toxicol 91(3) : 1335,"[Raymond Reif, Ahmed Ghallab, Lynette Beattie,...","{'data': [{'id': '7', 'type': 'projects'}]}"
9,The ascending pathophysiology of cholestatic l...,Hepatology,2017-02-01,Hepatology 65(2) : 722,"[Peter L.M. Jansen, Ahmed Ghallab, Nachiket Va...","{'data': [{'id': '7', 'type': 'projects'}]}"


## Create a publication "report"

In [24]:
project_cache = dict()

def get_project(pid):
    """ Get project information, with caching. """
    if not pid in project_cache:
        reponse = requests.get(SEEK_URL + f'projects/{pid}.json')
        json = r.json()
        projectCache[ids] = fullJson['data']
    
    return project_cache[pid]

def createProjectReport(projectData):
    for project in projectData['data']:
        pid =  project['id']
        pdata = get_project(pid)
        print(pdata)
        title = pdata['attributes']['title'];
        print("Project: title")
    
def create_report(pub_json):
    print()
    print('------------------------------------------------------------------------------')
    for key in ['title', 'authors', 'citation', 'published_data', 'journal']:
        print(pub_json.get(key))
    
    # project information for publication
    projects = pub_json['projects']
    createProjectReport(projects)


SyntaxError: invalid syntax (<ipython-input-24-61dfc90bcb1e>, line 6)

In [23]:
# now iterate over all publications
# and create a small report for each of them
for contentHash in publications:
    createReport(contentHash)


------------------------------------------------------------------------------
Specifications of Standards in Systems and Synthetic Biology: Status and Developments in 2016
F. Schreiber, G. D. Bader, P. Gleeson, Martin Golebiewski, M. Hucka, N. Le Novere, C. Myers, D. Nickerson, B. Sommer, D. Walthemath
J Integr Bioinform. 2016 Dec 18;13(3):289. doi: 10.2390/biecoll-jib-2016-289.
J Integr Bioinform
2017-02-12
LiSyM Core Infrastructure and Management (LiSyM-PD)

------------------------------------------------------------------------------
FAIRDOMHub: a repository and collaboration environment for sharing systems biology research
K. Wolstencroft, Olga Krebs, J. L. Snoep, N. J. Stanford, F. Bacall, Martin Golebiewski, R. Kuzyakiv, Q. Nguyen, S. Owen, S. Soiland-Reyes, J. Straszewski, D. D. van Niekerk, A. R. Williams, L. Malmstrom, B. Rinn, Wolfgang Müller, C. Goble
Nucleic Acids Res. 2017 Jan 4;45(D1):D404-D407. doi: 10.1093/nar/gkw1032. Epub 2016 Nov 28.
Nucleic Acids Res
2016-12-03
L

AttributeError: 'NoneType' object has no attribute 'group'